# Machine Learning 2 - Neural Networks

In this lab, we will use simple Neural Networks to classify the images from the simplified CIFAR-10 dataset. We will compare our results with those obtained with Decision Trees and Random Forests.

Lab objectives
----
* Classification with neural networks
* Influence of hidden layers and of the selected features on the classifier results

In [1]:
from lab_tools import CIFAR10, evaluate_classifier, get_hog_image
        
dataset = CIFAR10('./CIFAR10')

Pre-loading training data
Pre-loading test data


We will use the *[Multi-Layer Perceptron](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)* implementation from scikit-learn, which is only available since version 0.18. You can check which version of scikit-learn is installed by executing this :

In [2]:
import sklearn
print(sklearn.__version__)

0.19.2


If you have version 0.17 or older, please update your scikit-learn installation (for instance, with the command *pip install scikit-learn==0.19.1* in the terminal or Anaconda prompt)

## !Normalize the dataset!

In [14]:
import numpy as np

def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data,axis=0)
    data = (data - mean)/std
    return data, mean, std

## Build a simple neural network

* Using the [MLPClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) from scikit-learn, create a neural network with a single hidden layer.
* Train this network on the CIFAR dataset.
* Using cross-validation, try to find the best possible parameters.

In [4]:
from sklearn.neural_network import MLPClassifier

mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), activation="relu", learning_rate_init=0.001, solver="adam", random_state=33, alpha=1e-4, tol=1e-3)

In [5]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(5, random_state=33)

total_score = 0
total_conf = 0

for train,test in kf.split(dataset.train['hog'], dataset.train['labels']):
    train_x, mean, std = normalize(dataset.train['hog'][train])
    train_y = dataset.train['labels'][train]
    mlp_classifier = mlp_classifier.fit(train_x, train_y)
    
    test_x = (dataset.train['hog'][test]-mean)/std
    test_y = dataset.train['labels'][test]
    accuracy, conf = evaluate_classifier(mlp_classifier, test_x, test_y)
    
    total_score = total_score + accuracy
    total_conf = total_conf + conf

In [6]:
total_score = total_score/5
total_conf = total_conf/5

print("Validation accuracy: %.2f %%"%total_score)
print("Confusion matrix:")
print(total_conf)

Validation accuracy: 79.63 %
Confusion matrix:
[[812.6 125.2  41.2]
 [128.4 745.6 128. ]
 [ 59.  129.2 830.8]]


## Add hidden layers to the network.

Try to change the structure of the network by adding hidden layers. Using cross-validation, try to find the best architecture for your network.

## Test for different learning rate :

In [17]:
k=5

In [22]:
kf = StratifiedKFold(k, random_state=33)

lr_list = [1e-4,1e-5]
total_score = {}
total_conf = {}

for lr in lr_list:
    mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), activation="relu", learning_rate_init=lr, solver="adam", random_state=33, alpha=1e-4, tol=1e-4)
    train_x, mean, std = normalize(dataset.train['hog'][train])
    train_y = dataset.train['labels'][train]
    mlp_classifier = mlp_classifier.fit(train_x, train_y)

    test_x = (dataset.train['hog'][test]-mean)/std
    test_y = dataset.train['labels'][test]
    accuracy, conf = evaluate_classifier(mlp_classifier, test_x, test_y)

    total_score[lr] = accuracy
    total_conf[lr] = conf

In [23]:
for key in total_score.keys():
    print("Validation accuracy for learning rate of", key, ": %.2f %%"%total_score[key])
    print("Confusion matrix for learning rate of", key, ":")
    print(total_conf[key])

Validation accuracy for learning rate of 0.0001 : 79.40 %
Confusion matrix for learning rate of 0.0001 :
[[833 131  50]
 [123 739 140]
 [ 44 130 810]]
Validation accuracy for learning rate of 1e-05 : 74.70 %
Confusion matrix for learning rate of 1e-05 :
[[804 153  81]
 [131 678 160]
 [ 65 169 759]]


## Test for different NN length :

In [75]:
kf = StratifiedKFold(k, random_state=33)

param_list = [(100,),(100,100,),(100,100,100,)]
total_score = {}
total_conf = {}

for param in param_list:
    mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), activation="relu", learning_rate_init=param, solver="adam", random_state=33, alpha=1e-4, tol=1e-4)
    train_x, mean, std = normalize(dataset.train['hog'][train])
    train_y = dataset.train['labels'][train]
    mlp_classifier = mlp_classifier.fit(train_x, train_y)

    test_x = (dataset.train['hog'][test]-mean)/std
    test_y = dataset.train['labels'][test]
    accuracy, conf = evaluate_classifier(mlp_classifier, test_x, test_y)

    total_score[param] = accuracy
    total_conf[param] = conf

In [76]:
for key in total_score.keys():
    print("Validation accuracy for learning rate of", key, ": %.2f %%"%total_score[key])
    print("Confusion matrix for learning rate of", key, ":")
    print(total_conf[key])

Validation accuracy for learning rate of (100,) : 75.20 %
Confusion matrix for learning rate of (100,) :
[[790.2 157.8  66. ]
 [145.6 678.8 147. ]
 [ 64.2 163.4 787. ]]
Validation accuracy for learning rate of (100, 100) : 76.37 %
Confusion matrix for learning rate of (100, 100) :
[[793.2 141.4  53.2]
 [145.2 706.2 155.2]
 [ 61.6 152.4 791.6]]
Validation accuracy for learning rate of (100, 100, 100) : 76.90 %
Confusion matrix for learning rate of (100, 100, 100) :
[[798.  141.6  54. ]
 [141.8 710.  147. ]
 [ 60.2 148.4 799. ]]


## Test for different size of layers :

In [28]:
kf = StratifiedKFold(k, random_state=33)

param_list = [(100,1000)]
total_score = {}
total_conf = {}

for param in param_list:
    mlp_classifier = MLPClassifier(hidden_layer_sizes=param, activation="relu", learning_rate_init=1e-4, solver="adam", random_state=33, alpha=1e-4, tol=1e-4)
    train_x, mean, std = normalize(dataset.train['hog'][train])
    train_y = dataset.train['labels'][train]
    mlp_classifier = mlp_classifier.fit(train_x, train_y)

    test_x = (dataset.train['hog'][test]-mean)/std
    test_y = dataset.train['labels'][test]
    accuracy, conf = evaluate_classifier(mlp_classifier, test_x, test_y)

    total_score[param] = accuracy
    total_conf[param] = conf

In [27]:
for key in total_score.keys():
    print("Validation accuracy for learning rate of", key, ": %.2f %%"%total_score[key])
    print("Confusion matrix for learning rate of", key, ":")
    print(total_conf[key])

Validation accuracy for learning rate of (100, 1000) : 78.50 %
Confusion matrix for learning rate of (100, 1000) :
[[828 140  51]
 [130 725 147]
 [ 42 135 802]]


## Test for different size of layers :

In [37]:
kf = StratifiedKFold(k, random_state=33)

param_list = [(1000,)]
total_score = {}
total_conf = {}

for param in param_list:
    mlp_classifier = MLPClassifier(hidden_layer_sizes=param, activation="relu", learning_rate_init=1e-4, solver="adam", random_state=33, alpha=1e-4, tol=1e-4)
    train_x, mean, std = normalize(dataset.train['hog'][train])
    train_y = dataset.train['labels'][train]
    mlp_classifier = mlp_classifier.fit(train_x, train_y)

    test_x = (dataset.train['hog'][test]-mean)/std
    test_y = dataset.train['labels'][test]
    accuracy, conf = evaluate_classifier(mlp_classifier, test_x, test_y)

    total_score[param] = accuracy
    total_conf[param] = conf

In [38]:
for key in total_score.keys():
    print("Validation accuracy for learning rate of", key, ": %.2f %%"%total_score[key])
    print("Confusion matrix for learning rate of", key, ":")
    print(total_conf[key])

Validation accuracy for learning rate of (1000,) : 81.90 %
Confusion matrix for learning rate of (1000,) :
[[849 116  40]
 [109 773 125]
 [ 42 111 835]]
